#In this section: applimentation of transformer(pipes_), separation the target variable, scaling and standardization, division data into test and training. Applimentation of GridSearchCV (ML with Teacher). Usage of Grid search optimization algorithm which lets us select the best parameters for our optimization problem.

Also in this section was used Automatic parameter searches via RandomizedSearchCV.
RandomizedSearchCV randomly passes the set of hyperparameters and calculate the score and gives the best set of hyperparameters which gives the best score as an output.

In [1]:
import pandas as pd
import pipes_ as pp
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score
from matplotlib import pyplot as plt


# 1.We review the information. Preparing data for processing format

In [2]:
df_ = pd.read_csv('NewDataFrame.csv')
df_.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df_.drop(["a"], axis=1, inplace=True)

In [3]:
df_ = pp.myPipe.transform(df_)

In [5]:
X = df_.drop(['CreditScore','PaymentBehaviour'], axis = 1)

In [7]:
Y = df_['CreditScore']

# Создание проверочного набора (Create a validation set)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.4, random_state = 0)

In [9]:
X_train.shape, Y_train.shape

((60000, 41), (60000,))

# a. ML With Teacher

# GridSearchCV 

GridSearchCV is the process of performing hyperparameter tuning in order to determine the optimal values for a given model. As mentioned above, the performance of a model significantly depends on the value of hyperparameters.

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
# Поиск гиперпараметров 
# проведем настройку гиперпараметров для дерева и ближайших соседей
#we will adjust the hyperparameters for the tree and nearest neighbors;
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

tree = DecisionTreeClassifier(max_depth=7, min_samples_split=2, min_samples_leaf=1,random_state=0)
knn = KNeighborsClassifier(n_neighbors=3)

tree.fit(X_train, Y_train)
knn.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

In [12]:
# Оценим точность
#Estimate the accuracy
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [16]:
tree_pred = tree.predict(X_test)
roc_auc_score (Y_test, tree_pred,multi_class='ovo'), accuracy_score (Y_test, tree_pred,multi_class='ovo' )

AxisError: axis 1 is out of bounds for array of dimension 1

In [24]:
knn_pred = knn.predict(X_test)
roc_auc_score (Y_test, knn_pred), accuracy_score (Y_test, knn_pred)
# accuracy_score - оцениваем точность попадания в класс
# roc_auc_score - оцениваем площать под кривой

(0.751015459716468, 0.7995)

In [25]:
# поиск для дерева
# формируем словарь
tree_params = {'max_depth': range(1,11), 'max_features': range(4,19)}

In [26]:
tree_grid = GridSearchCV(tree, tree_params,cv=5, verbose=True)

In [27]:
tree_grid.fit(X_train, Y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=7, random_state=0),
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(4, 19)},
             verbose=True)

In [28]:
tree_grid.best_params_

{'max_depth': 9, 'max_features': 18}

In [29]:
tree_grid.best_score_

0.82652

In [30]:
pred = tree_grid.predict(X_test)

In [31]:
roc_auc_score(Y_test, pred), accuracy_score(Y_test, pred)

(0.7711669728346988, 0.82654)

In [ ]:
# поиск для соседей

In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [140]:
sc = StandardScaler()

In [141]:
sc.fit(X_train)
X = sc.transform(X_train)
X_test = sc.transform(X_test)

In [142]:
knn_params= {'n_neighbors':range(1,11)}

In [144]:
knn_grid = GridSearchCV(knn, knn_params,cv=5, verbose=True)

In [146]:
#knn_grid.fit(X_train,Y_train)

In [ ]:
knn_grid.score(X_train,Y_train)

In [ ]:
pre = knn_grid.predict(X_test)

In [ ]:
roc_auc_score(Y_test, pre), accuracy_score(Y_test, pre)

# Automatic parameter searches


RandomizedSearchCV randomly passes the set of hyperparameters and calculate the score and gives the best set of hyperparameters which gives the best score as an output.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [10]:
#ЗАДАЕМ 20 ПАРАМЕТРОВ 
# define the parameter space that will be searched over
param_distributions = {'n_estimators': randint(1, 20),
                        'max_depth': randint(5, 10)}

In [11]:
# now create a searchCV object and fit it to the data
search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=0),
                             n_iter=5,
                             param_distributions=param_distributions,
                             random_state=0)

search.fit(X_train, Y_train)
search.best_params_

{'max_depth': 9, 'n_estimators': 16}

In [12]:
# the search object now acts like a normal random forest estimator
# with max_depth=9 and n_estimators=16 -ABOVE
search.score(X_test, Y_test)

0.82254